In [35]:
!pip install transformer_lens
!pip install circuitsvis
!pip install torchtyping
!pip insatll einops

ERROR: unknown command "insatll" - maybe you meant "install"


In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in c

In [1]:
# %%
import torch
import numpy as np
import random
from transformer_lens import HookedTransformer, ActivationCache
import transformer_lens.utils as utils
import plotly.express as px

torch.set_grad_enabled(False)

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)


# %%
model = HookedTransformer.from_pretrained('gemma-7b', dtype=torch.float16)
print(model)

# %%
out = model.generate("Hi, my name is")
print(out)

# %%

n_layers = model.cfg.n_layers
d_model = model.cfg.d_model
n_heads = model.cfg.n_heads
d_head = model.cfg.d_head
d_mlp = model.cfg.d_mlp
d_vocab = model.cfg.d_vocab


# %%
prelude = "A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. "
question = "To prevent desiccation and injury, the embryos of terrestrial vertebrates are encased within a fluid secreted by the:"
answers = """
(A) amnion
(B) chorion
(C) allantois
(D) yolk sac
"""
post_text = "Answer: ("

answer_token = "A"

text = prelude + question + answers + post_text
print(model.to_str_tokens(text))



# %%

utils.test_prompt(text, answer_token, model, prepend_space_to_answer=False)


# %%
# DLA

logits, cache = model.run_with_cache(text)
cache: ActivationCache


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to s

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded pretrained model gemma-7b into HookedTransformer
HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (blocks): ModuleList(
    (0-27): 28 x TransformerBlock(
      (ln1): RMSNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): RMSNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): GroupedQueryAttention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
        (hook_rot_k): HookPoint()
        (hook_rot_q): HookPoint()
      )
      (mlp): GatedMLP(
        (hook_pre): HookPoint()
        (hook_pre_linear): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): Hook

  0%|          | 0/10 [00:00<?, ?it/s]

Hi, my name is Iain Lees~Jones, and I am very happy
['<bos>', 'A', ' highly', ' knowledgeable', ' and', ' intelligent', ' AI', ' answers', ' multiple', '-', 'choice', ' questions', ' about', ' Biology', '.', ' To', ' prevent', ' desic', 'cation', ' and', ' injury', ',', ' the', ' embryos', ' of', ' terrestrial', ' vertebrates', ' are', ' encased', ' within', ' a', ' fluid', ' secreted', ' by', ' the', ':', '\n', '(', 'A', ')', ' am', 'nion', '\n', '(', 'B', ')', ' chor', 'ion', '\n', '(', 'C', ')', ' all', 'anto', 'is', '\n', '(', 'D', ')', ' yolk', ' sac', '\n', 'Answer', ':', ' (']
Tokenized prompt: ['<bos>', 'A', ' highly', ' knowledgeable', ' and', ' intelligent', ' AI', ' answers', ' multiple', '-', 'choice', ' questions', ' about', ' Biology', '.', ' To', ' prevent', ' desic', 'cation', ' and', ' injury', ',', ' the', ' embryos', ' of', ' terrestrial', ' vertebrates', ' are', ' encased', ' within', ' a', ' fluid', ' secreted', ' by', ' the', ':', '\n', '(', 'A', ')', ' am', 'nion

Performance on answer token:
Rank: 0        Logit: 25.88 Prob: 90.04% Token: |A|

Top 0th token. Logit: 25.88 Prob: 90.04% Token: |A|
Top 1th token. Logit: 22.75 Prob:  3.96% Token: |B|
Top 2th token. Logit: 22.27 Prob:  2.44% Token: |C|
Top 3th token. Logit: 21.83 Prob:  1.57% Token: |D|
Top 4th token. Logit: 21.69 Prob:  1.37% Token: |a|
Top 5th token. Logit: 18.95 Prob:  0.09% Token: |1|
Top 6th token. Logit: 18.94 Prob:  0.09% Token: |b|
Top 7th token. Logit: 18.78 Prob:  0.07% Token: |c|
Top 8th token. Logit: 18.53 Prob:  0.06% Token: | A|
Top 9th token. Logit: 18.53 Prob:  0.06% Token: |d|


Ranks of the answer tokens: [('A', 0)]

In [2]:
from datasets import load_dataset

dataset = load_dataset("cais/mmlu", "high_school_biology")

In [3]:
prompts = []
correct_answers = []

letters = ['A', 'B', 'C', 'D']

prelude = "A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. "
post_text = "Answer: ("

for i in range(100):
  question = dataset["test"][i]["question"] + ':\n'

  answers = f"""
(A) {dataset["test"][i]["choices"][0]}
(B) {dataset["test"][i]["choices"][1]}
(C) {dataset["test"][i]["choices"][2]}
(D) {dataset["test"][i]["choices"][3]}
"""

  correct_answers.append(letters[dataset["test"][i]["answer"]])

  prompts.append(prelude + question + answers + post_text)

In [4]:
print(prompts[0])

A highly knowledgeable and intelligent AI answers multiple-choice questions about Biology. In a population of giraffes, an environmental change occurs that favors individuals that are tallest. As a result, more of the taller individuals are able to obtain nutrients and survive to pass along their genetic information. This is an example of:

(A) directional selection.
(B) stabilizing selection.
(C) sexual selection.
(D) disruptive selection.
Answer: (


In [5]:
batch_size = 1

stacked_components = []

for b in range(0, 100, batch_size):
  _, cache = model.run_with_cache(model.to_tokens(prompts[b:b+batch_size]))

  decomposed, component_labels = cache.decompose_resid(layer=-1, return_labels=True, pos_slice=-1)
  stacked_mlps = decomposed[2::2, :, :]
  component_labels = component_labels[2::2]

  stacked_heads, head_labels = cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)

  stacked_components.append(torch.cat([stacked_mlps, stacked_heads]))

component_labels.extend(head_labels)
stacked_components = torch.cat(stacked_components, dim=1)

Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now
Tried to stack head results when they weren't cached. Computing head results now


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 22.17 GiB of which 10.88 MiB is free. Process 176286 has 22.15 GiB memory in use. Of the allocated memory 21.84 GiB is allocated by PyTorch, and 83.11 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [92]:
stacked_components.shape

torch.Size([476, 1, 3072])

In [12]:
decomposed, component_labels = cache.decompose_resid(layer=-1, return_labels=True, pos_slice=-1)
stacked_mlps = decomposed[2::2, :, :]
component_labels = component_labels[2::2]

stacked_heads, head_labels = cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)

stacked_components = torch.cat([stacked_mlps, stacked_heads])
component_labels.extend(head_labels)

In [13]:
stacked_components.shape

torch.Size([476, 1, 3072])

In [27]:
letter_tokens = model.to_tokens(['A', 'B', 'C', 'D'], prepend_bos=False)[:, 0]

dla_letters = cache.logit_attrs(stacked_components, letter_tokens).to('cpu')

print('dla shape', dla_letters.shape)


dla shape torch.Size([476, 4])


In [28]:
logit_attr = dla_letters[:, 0] - dla_letters.mean(1)

In [49]:
top_k_values, top_k_indices = torch.topk(logit_attr, 10)
top_labels = [component_labels[i] for i in top_k_indices]

In [31]:
fig = px.bar(y=top_k_values, x=top_labels)
fig.show()

In [55]:
component_labels[28]

'L0H0'

In [41]:
top_k_indices

tensor([ 40,  43,  31,  26,  38,  22, 458,  25,  21, 430])

In [38]:
from torchtyping import TensorType as TT
import einops

def stack_head_pattern_from_cache(
        cache,
    ) -> TT["layer_and_head_index", "batch", "dest_pos", "src_pos"]:
        """Stacks the head patterns from the cache into a single tensor."""
        stacked_head_pattern = torch.stack(
            [cache["pattern", l] for l in range(model.cfg.n_layers)], dim=0
        )
        stacked_head_pattern = einops.rearrange(
            stacked_head_pattern,
            "layer batch head_index dest_pos src_pos -> (layer head_index) batch dest_pos src_pos",
        )
        return stacked_head_pattern

In [43]:
patterns = stack_head_pattern_from_cache(cache).cpu()

In [62]:
plot_toks = [f"{tok} ({i})" for i, tok in enumerate(model.to_str_tokens(text))]
fig = px.imshow(patterns[458-28, 0], x=plot_toks, y=plot_toks, aspect='auto')
fig.update_layout(width=1400, height=1400)

In [71]:
letter_idxs = [38, 44, 50, 57]

plot_toks = [f"{tok} ({i})" for i, tok in enumerate(model.to_str_tokens(text))]
fig = px.imshow(patterns[458-28, 0, -1, None], x=plot_toks, aspect='auto')
fig.update_layout(width=1400, height=400)

In [ ]:
top_k_values, top_k_indices = torch.topk(dla, 10)

In [ ]:
_, large_mag_indices = torch.topk(torch.abs(dla), 20)
large_mag_values = dla[large_mag_indices]

In [ ]:
top_labels = [component_labels[i] for i in top_k_indices]

In [ ]:
fig = px.bar(y=top_k_values, x=top_labels)
fig.show()

In [ ]:
top_labels = [component_labels[i] for i in large_mag_indices]
fig = px.bar(y=large_mag_values, x=top_labels)
fig.show()

In [ ]:
correct_dir = # unembed(anwer) - mean(unembed(all))